In [1]:
from sklearn.feature_selection import SelectKBest, f_classif
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tslearn.utils import to_time_series_dataset
import biosppy.signals.ecg as ecg
import matplotlib.pyplot as plt
import neurokit2 as nk
import heartpy as hp
from statistics import mean
import functions
from multiprocessing import Pool
from datetime import time
import functions
import scipy
import math
import operator
from functions import *
import itertools

In [2]:
folder = ""
raw = folder + "raw_data/"
# raw_dataset_x = pd.read_csv(raw + 'X_train.csv',index_col=['id'])


In [3]:
# raw_dataset_x.info()

In [4]:
# raw_dataset_y = pd.read_csv(raw + 'y_train.csv',index_col=['id'])
# raw_dataset_y.head()

In [5]:
#x_train, x_test, y_train, y_test = train_test_split(raw_dataset_x, raw_dataset_y, test_size=0.2, random_state=100)
x_train = pd.read_csv(raw + 'X_train.csv', index_col=['id'])
x_test = pd.read_csv(raw + 'X_test.csv', index_col=['id'])
y_train = pd.read_csv(raw + 'y_train.csv', index_col=['id'])
y_test = pd.DataFrame({'A': []})

In [6]:
count_0 = 0
def apply_func(index,row):
    #print("Extracting features from row -> ",index)
    a = datetime.datetime.now()
    ecg_signal = row.dropna().to_numpy()
    ecg_cleaned = nk.ecg_clean(ecg_signal, sampling_rate=300)
    #     ecg_cleaned = ecg_signal

    # to try
    #max_abs = max(ecg_cleaned.min(), ecg_cleaned.max(), key=abs)
    #ecg_cleaned = ecg_cleaned / max_abs
    #     # Extract R-peaks locations
    peaks, rpeaks = nk.ecg_peaks(ecg_cleaned, sampling_rate=300)
    # Delineate the ECG signal
    b = datetime.datetime.now()
    #print("r_peak", (b - a).seconds)
    values_dict = {}
    nk_features = np.zeros(42)
    if len(rpeaks['ECG_R_Peaks']) > 3:
        try:
            signal_dwt, waves_dwt = nk.ecg_delineate(ecg_cleaned, rpeaks, show_type='all', sampling_rate=300,
                                                     method="dwt")
            for key, arr in waves_dwt.items():
                if 'Onsets' in key or 'Offsets' in key:
                    array = np.array(arr)
                    #row = add_basic_info(row, np.array(arr), key)
                    #row = add_basic_info(row, amplitudes, f"{key}_amplitudes")
            nk_features = get_nk_features(ecg_cleaned, rpeaks['ECG_R_Peaks'], waves_dwt)
        except:
            print("Signal flipped")
            ecg_signal = row.dropna().to_numpy()
            ecg_signal = -ecg_signal
            ecg_cleaned = nk.ecg_clean(ecg_signal, sampling_rate=300)
            peaks, rpeaks = nk.ecg_peaks(ecg_cleaned, sampling_rate=300)
            signal_dwt, waves_dwt = nk.ecg_delineate(ecg_cleaned, rpeaks, show_type='all', sampling_rate=300,
                                                     method="dwt")
            for key, arr in waves_dwt.items():
                if 'Onsets' in key or 'Offsets' in key:
                    array = np.array(arr)
                    #row = add_basic_info(row, np.array(arr), key)
            nk_features = get_nk_features(ecg_cleaned, rpeaks['ECG_R_Peaks'], waves_dwt)
    global count_0 
    count_0 += 42 - np.count_nonzero(nk_features)
    mu = np.zeros(180)
    median = np.zeros(180)
    std = np.zeros(180)
    if len(rpeaks['ECG_R_Peaks']) >= 2:
        # print(ecg.extract_heartbeats(signal, r_peaks, 300))

        beats = ecg.extract_heartbeats(ecg_signal, rpeaks['ECG_R_Peaks'], 300)['templates']

        if len(beats) != 0:
            mu = np.mean(beats, axis=0)
            median = np.median(beats, axis=0)
            std = np.std(beats, axis=0)

    c = datetime.datetime.now()
    #print("extract_heartbeats", (c - b).seconds)

    d = datetime.datetime.now()
    #print("delineate", (d - c).seconds)
    # fft_window = np.fft.fft(ecg_cleaned)
    # freq = np.fft.fftfreq(np.array(ecg_cleaned).shape[-1], d=1/300)
    # fft_ps = np.abs(fft_window)**2
    
    #print(fft_window, freq, fft_ps)
    e = datetime.datetime.now()
    #print("fft", (e - d).seconds)
    row = row.drop(columns=[f'x{num}' for num in range(17807)])
    f = datetime.datetime.now()
    
    #print("drop", (f - e).seconds)
    for i in range(180):
        values_dict[f'mu_{i}'] = mu[i]
        values_dict[f'median_{i}'] = median[i]
        values_dict[f'std_{i}'] = std[i]
    # for i in range(len(fft_window)):
    #     #values_dict[f'fft_window_{i}'] = fft_window[i]
    #     #values_dict[f'freq_{i}'] = freq[i]
    #     #values_dict[f'fft_ps_{i}'] = fft_ps[i]
    #     pass
    for i in range(len(nk_features)):
        values_dict[f'nk_feat_{i}'] = nk_features[i]
    
    #row.append(pd.Series(values_dict))
    g = datetime.datetime.now()
    #print("add", (g - f).seconds)
    return values_dict

In [7]:
#classify ecg
from functions import *
def classify_ecg(x_train):
    x_train_ecg = x_train.copy()
    '''
    x_train_ecg = x_train_ecg.apply(apply_func)
    '''
    new_train_dict = {}
    for index,row in x_train.iterrows():
        new_train_dict[str(index)] = apply_func(index,row)
        if len(new_train_dict) % 500 == 0:
            print(len(new_train_dict))
        
    return pd.DataFrame.from_dict(new_train_dict,orient='index')
    
    return x_train_ecg

In [8]:
x_train = classify_ecg(x_train)
print("Number of zeros -> ",count_0)

C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _metho

Signal flipped


C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _metho

500


C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _metho

...

5000


C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


Signal flipped


C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _metho

Number of zeros ->  541


In [9]:
x_train.to_csv('x_train_preprocess.csv')

In [10]:
x_test = classify_ecg(x_test)

C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _metho

500


C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _metho

1000


C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _metho

...

C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _metho

2500


C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _metho

3000


C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _metho

In [ ]:
x_test.to_csv('x_test_preprocess.csv')

In [ ]:
#count nan values per column
def count_nan(df):
    for col in df.columns:
        if(df[col].isna().sum() > 0):
            print(col,df[col].isna().sum())
    
count_nan(x_train)